In [27]:
import pandas as pd
import numpy as np
import re

pd.set_option('max_colwidth', 100)
pd.set_option('display.width', 2000)

In [3]:
file1 = open('../data/train.txt', 'r') 
lines = file1.readlines()

In [4]:
ids = []
revs = []
comps = []
txts = []

for line in lines:
    ids.append(line[1:5])
    revs.append(line[6:9])
    comps.append(line[10:13])
    txts.append(line[15:])

In [5]:
df = pd.DataFrame({"Avis" : revs, "Entreprise" : comps, "Tweet" : txts}, index = ids)
df.head(10)

,Avis,Entreprise,Tweet
0000,neu,apl,20 min line @apple store @short pump.\n
0001,irr,msf,Nueva tecnología convierte cualquier superfici...
0002,neu,ggl,Some people should not post replies in #Google...
0003,neg,apl,I know a few others having same issue RT @Joel...
0004,neg,msf,"#Microsoft - We put the """"backwards"""" into bac..."
0005,neg,twt,#twitter is sooo trash ritenow with all dezz #...
0006,neu,apl,RT @jesperordrup: Hi @apple. Household has 4 i...
0007,irr,msf,#ALG Culminando formación en #Microsoft M-2778...
0008,neu,msf,#Microsoft Community Blogs The 7/365 Review - ...
0009,irr,twt,Buenas noches a todos #Twitter off\n


In [6]:
df.Avis.unique()

array(['neu', 'irr', 'neg', 'pos'], dtype=object)

In [7]:
df.Entreprise.unique()

array(['apl', 'msf', 'ggl', 'twt'], dtype=object)

In [8]:
len(df)

4173

In [9]:
df.groupby(by="Avis").count()

,Entreprise,Tweet
Avis,,
irr,1389,1389
neg,486,486
neu,1871,1871
pos,427,427


In [10]:
df.groupby(by=["Entreprise", "Avis"]).count()

Tweet
Entreprise Avis       
apl        irr     125
           neg     267
           neu     422
           pos     140
ggl        irr     398
           neg      47
           neu     455
           pos     167
msf        irr     401
           neg     111
           neu     516
           pos      73
twt        irr     465
           neg      61
           neu     478
           pos      47

In [29]:
df_with_hashtags = df[df.Tweet.str.contains("#")]
print(len(df_with_hashtags))
df_with_hashtags.head(10)

3583


,Avis,Entreprise,Tweet
0001,irr,msf,Nueva tecnología convierte cualquier superficie en una pantalla multitactil. http://t.co/EDibLL5...
0002,neu,ggl,Some people should not post replies in #Google+ threads. Their posts only continue to weaken the...
0004,neg,msf,"#Microsoft - We put the """"backwards"""" into backwards compatibility. #instantfollowback\n"
0005,neg,twt,#twitter is sooo trash ritenow with all dezz #highscoolmemories -__-\n
0007,irr,msf,#ALG Culminando formación en #Microsoft M-2778 y en #IndicadoresDeGestión. Gracias a nuestros cl...
0008,neu,msf,#Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business http://t.co/gPB2MjFW\n
0009,irr,twt,Buenas noches a todos #Twitter off\n
0010,neu,apl,This video brought a small tear to my eye http://t.co/3va2ninp. Watch to @SteveWoz opinion of #S...
0011,neg,msf,"I gave a try to Windows 8 Dev. Preview and it was good but the UI is way too messy right now, wa..."
0012,neu,ggl,now watching live stream of #Google new mobile #Nexus\n


In [26]:
df_with_mentions = df[df.Tweet.str.contains("@")]
print(len(df_with_mentions))
df_with_mentions.head(10)

1908


,Avis,Entreprise,Tweet
0000,neu,apl,20 min line @apple store @short pump.\n
0003,neg,apl,I know a few others having same issue RT @Joelplane: 9% now on my second full charge of the day....
0006,neu,apl,"RT @jesperordrup: Hi @apple. Household has 4 iphones, 2 ipads, 2 minis, 2 apple tv, AirPorts, Ti..."
0010,neu,apl,This video brought a small tear to my eye http://t.co/3va2ninp. Watch to @SteveWoz opinion of #S...
0014,pos,apl,@apple..good ipad\n
0015,neu,apl,@BikerVox @mariomussolini @ios_notch @apple xD thats why i have a back up PC… just incase *hides...
0017,pos,msf,"@RAD_Software Yes good points about #ERP but unfair assessment in saying """"#SAGE, #SAP and #Micr..."
0023,neg,twt,RT @nagoul1: #Retweets section of my #twitter account has not been working for 17 hours! Is Ever...
0025,neg,apl,"Comparing @Office #live with @google #apps and @apple #iCloud , I do not see #iCloud leading the..."
0026,irr,ggl,@iley_little we'lll figure it out tm in gym #google\n


In [149]:
#hashtag_re = r"#([^\s,]*)"
hashtag_re = r"#([\w\+_-]*)"
mention_re = r"@(\w*)"
link_re = r"(https?://|(https?://)?bit.ly)[^\s]+"

#preprocessing
def remove_slash_n(tweet):
    return tweet[:-1] if(tweet.endswith("\n")) else tweet
    
def replace_links(tweet, rep="_LINK_"):
    return re.sub(link_re, rep, tweet)

def remove_hashtags_and_mentions(tweet):
    s = re.sub(hashtag_re, r"\1", tweet)
    return re.sub(mention_re, r"\1", s)

def preprocess_tweet(tweet):
    return replace_links(remove_slash_n(remove_hashtags_and_mentions(tweet))) if tweet != None else None

#finding list of hashtags & mentions
def find_hashtags(tweet):
    return [h for h in re.findall(hashtag_re, tweet) if h != ""]

def find_mentions(tweet):
    return [h for h in re.findall(mention_re, tweet) if h != ""]

In [144]:
re.findall(r'@([^\s,]*)', "hello @worldéç/ù, @Test")

['worldéç/ù', 'Test']

In [145]:
re.findall(hashtag_re, "hello #world-é#ç/ù, #Test")

['world-é', 'ç', 'Test']

In [146]:
re.sub(r"https?://[^\s]+", "_LINK_", "http://t.co/3va2ninp. hello @worldéçù @Test")

'_LINK_ hello @worldéçù @Test'

In [147]:
re.sub(r"@(\w*)", r"\1", "hello @worldéç/ù @Test")

'hello worldéç/ù Test'

In [148]:
preprocess_tweet("20 min line @apple store @short pump.")

'20 min line apple store short pump.'

In [150]:
df = pd.read_pickle("../data/df.pk")

df["Hashtags"] = df.Tweet.apply(find_hashtags)
df["Mentions"] = df.Tweet.apply(find_mentions)
df["Tweet"] = df.Tweet.apply(preprocess_tweet)
df.head(10)

,Avis,Entreprise,Tweet,Hashtags,Mentions
0000,neu,apl,20 min line apple store short pump.,[],"[apple, short]"
0001,irr,msf,Nueva tecnología convierte cualquier superficie en una pantalla multitactil. _LINK_ Microsoft om...,"[Microsoft, omnitouch]",[]
0002,neu,ggl,Some people should not post replies in Google+ threads. Their posts only continue to weaken thei...,[Google+],[]
0003,neg,apl,I know a few others having same issue RT Joelplane: 9% now on my second full charge of the day. ...,[],"[Joelplane, Apple]"
0004,neg,msf,"Microsoft - We put the """"backwards"""" into backwards compatibility. instantfollowback","[Microsoft, instantfollowback]",[]
0005,neg,twt,twitter is sooo trash ritenow with all dezz highscoolmemories -__-,"[twitter, highscoolmemories]",[]
0006,neu,apl,"RT jesperordrup: Hi apple. Household has 4 iphones, 2 ipads, 2 minis, 2 apple tv, AirPorts, Time...",[],"[jesperordrup, apple]"
0007,irr,msf,ALG Culminando formación en Microsoft M-2778 y en IndicadoresDeGestión. Gracias a nuestros clien...,"[ALG, Microsoft, IndicadoresDeGestión]",[]
0008,neu,msf,Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business _LINK_,[Microsoft],[]
0009,irr,twt,Buenas noches a todos Twitter off,[Twitter],[]


In [153]:
df[df.Tweet.str.contains("/")]

,Avis,Entreprise,Tweet,Hashtags,Mentions
0008,neu,msf,Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business _LINK_,[Microsoft],[]
0044,neg,apl,So PhoenixSwinger 's iPhone 4 is giving her a hella hard time w/ the iOS5 update apple,[],"[PhoenixSwinger, apple]"
0081,neu,apl,"azee1v1 apple umber Better file info/easier to get to, duplication of files when importing. Cate...",[],"[azee1v1, apple, umber]"
0099,irr,msf,"Job ICT Sachbearbeiter Immobilienbewirtschaftung (m/w): Gfeller Treuhand und Verwaltungs AG, Düb...","[Job, ICT, Microsoft, IT]",[]
0156,neu,twt,iamRoyaltyFRESH Pls let us know if you need help w/ music marketing promo. We got Twitter & Yout...,"[Twitter, Youtube]",[iamRoyaltyFRESH]
0185,neg,apl,"Why doesn't apple iCloud sync Stickies? They've always been around, just nothing every progresse...",[],"[apple, gruber]"
0188,neg,apl,anyone else stuck in duped calendar/mail/battery sucking apple hell? iCloud and OSX Lion are a d...,"[iCloud, OSX]",[apple]
0227,pos,ggl,There you go - according to _LINK_ Google+ is builtin to the OS/Nexus line? Smart move :),[Google+],[]
0235,pos,apl,S/O to apple for replacing my phone for free,[],[apple]
0249,neu,msf,Streaming News 24/7 at _LINK_ - Ballmer feels lucky Microsoft didn't buy Yahoo in 2008 _LINK_,"[Microsoft, Yahoo]",[]


In [154]:
df.to_pickle("../data/df_preproc.pk")